In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import benchlingapi
import sqlalchemy
from sqlalchemy import select
import xmltodict

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.registry as registry

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Config

In [ ]:
lib_parts = reg[("LIB", "parts")]
olib_oligos = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
part_types = reg[("LIB", "parts", "Part types")]

# Test

In [ ]:
db_url = sqlalchemy.engine.URL.create(
    **{"drivername": "cockroachdb", **config["geneious_test"]}
)
engine = sqlalchemy.create_engine(db_url)
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
db_url.render_as_string(False)

In [ ]:
metadata = sqlalchemy.MetaData()
metadata.reflect(bind=engine)

In [ ]:
conn = engine.connect()

In [ ]:
metadata.tables.keys()

In [ ]:
for name, t in metadata.tables.items():
    print(name)
    q = select([t]).limit(20)
    rows = conn.execute(q).all()
    for row in rows:
        print(row)
    print()

In [ ]:
q = select([metadata.tables["annotated_document"]])
rows = conn.execute(q).all()

In [ ]:
docs = [xmltodict.parse(row["document_xml"], dict_constructor=dict) for row in rows]

In [ ]:
BAD_CLASSES = [
    "com.biomatters.geneious.publicapi.implementations.sequence.OligoSequenceDocument",
    "com.biomatters.geneious.publicapi.documents.OperationRecordDocument",
]
good_docs = [d for d in docs if d["document"]["@class"] not in BAD_CLASSES]

In [ ]:
set(d["document"]["@class"] for d in good_docs)

In [ ]:
d[30]

In [ ]:
x = xmltodict.parse(row["document_xml"], dict_constructor=dict)

In [ ]:
x

In [ ]:
t = metadata.tables["g_user_group_role"]

In [ ]:
x = select([t])  # .limit(1)

In [ ]:
t.columns.keys()

In [ ]:
conn.execute(x).all()

In [ ]:
conn.execute(x).one()

In [ ]:
select([metadata.tables["folder"]]).

In [ ]:
s.keys()

In [ ]:
x.

In [ ]:
y = x.select()

## Folders

In [ ]:
folder_table = metadata.tables["folder"]

In [ ]:
q = select([folder_table]).where(folder_table.c.visible == True)
rows = conn.execute(q).all()

In [ ]:
rows

In [ ]:
rows[0].items()

In [ ]:
folder_table

In [ ]:
x = metadata.tables["folder"].columns.g_group_id

In [ ]:
y = next(iter(x.foreign_keys))

In [ ]:
metadata.tables["folder"].columns.g_group_id.foreign_keys

In [ ]:
metadata.tables["folder"].columns.g_group_id

In [ ]:
for t in metadata.tables.values():
    # print(repr(t))
    print(
        t.name,
        [
            (c.name, next(iter(c.foreign_keys)).target_fullname)
            for c in t.columns
            if len(c.foreign_keys)
        ],
    )

In [ ]:
from sqlalchemy import Column, ForeignKey, Integer
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import relationship, backref  # , Integer, ForeignKey
from sqlalchemy.orm.collections import column_mapped_collection
import inflect
import re

Base = automap_base()

# FROM: https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html
_pluralizer = inflect.engine()


def pluralize_collection(base, local_cls, referred_cls, constraint):
    "Produce an 'uncamelized', 'pluralized' class name, e.g."
    "'SomeTerm' -> 'some_terms'"

    # print(base.__name__, local_cls.__name__, referred_cls.__name__)
    if local_cls.__name__ == "Folder" and referred_cls.__name__ == "Folder":
        return "children"

    referred_name = referred_cls.__name__
    uncamelized = re.sub(r"[A-Z]", lambda m: "_%s" % m.group(0).lower(), referred_name)
    pluralized = _pluralizer.plural(uncamelized)
    return pluralized


class Folder(Base):
    __tablename__ = "folder"

    # override schema elements like Columns
    # user_name = Column('name', String)

    # SEE: https://docs.sqlalchemy.org/en/14/orm/self_referential.html#self-referential
    # children = relationship("Folder", backref=backref('parent_folder', remote_side=[Folder.id]))
    id = Column(Integer, primary_key=True)
    parent_folder_id = Column(Integer, ForeignKey("folder.id"))
    #     children = relationship(
    #         "Folder", backref=backref("parent_folder", remote_side=[id])
    #     )
    # FROM: https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html
    children = relationship(
        "Folder",
        # cascade deletions
        cascade="all, delete-orphan",
        # many to one + adjacency list - remote_side
        # is required to reference the 'remote'
        # column in the join condition.
        backref=backref("parent_folder", remote_side=id),
        # children will be represented as a dictionary
        # on the "name" attribute.
        collection_class=column_mapped_collection(name),
    )


# class SpecialElement(Base):
#     __tablename__ = "special_element"

#     #folder = relationship("Folder", backref="special_elements")

# reflect
Base.prepare(
    engine, reflect=True, name_for_collection_relationship=pluralize_collection
)

# we still have Address generated from the tablename "address",
# but User is the same as Base.classes.User now
# Address = Base.classes.address

In [ ]:
stmt = select(Folder).limit(1)
res = session.execute(stmt).one()[0]

In [ ]:
res.folder

In [ ]:
stmt = select(Folder).where(Folder.visible == True)
result = session.execute(stmt)

In [ ]:
l = result.all()

In [ ]:
x = l[1][0]

In [ ]:
x.parent_folder.children

## Non-reflected

In [ ]:
from sqlalchemy import Column, ForeignKey, Integer
from sqlalchemy.orm import (
    declarative_base,
    relationship,
    backref,
)  # , Integer, ForeignKey
from sqlalchemy.orm.collections import column_mapped_collection

Base = declarative_base()


class Folder(Base):
    __tablename__ = "folder"

    # override schema elements like Columns
    # user_name = Column('name', String)

    # SEE: https://docs.sqlalchemy.org/en/14/orm/self_referential.html#self-referential
    # children = relationship("Folder", backref=backref('parent_folder', remote_side=[Folder.id]))
    id = Column(Integer, primary_key=True)
    parent_folder_id = Column(Integer, ForeignKey("folder.id"))
    #     children = relationship(
    #         "Folder", backref=backref("parent_folder", remote_side=[id])
    #     )
    # FROM: https://docs.sqlalchemy.org/en/14/orm/extensions/automap.html
    children = relationship(
        "Folder",
        # cascade deletions
        cascade="all, delete-orphan",
        # many to one + adjacency list - remote_side
        # is required to reference the 'remote'
        # column in the join condition.
        backref=backref("parent_folder", remote_side=id),
        # children will be represented as a dictionary
        # on the "name" attribute.
        collection_class=column_mapped_collection(name),
    )


# class SpecialElement(Base):
#     __tablename__ = "special_element"

#     #folder = relationship("Folder", backref="special_elements")

# reflect
Base.prepare(
    engine, reflect=True, name_for_collection_relationship=pluralize_collection
)

# we still have Address generated from the tablename "address",
# but User is the same as Base.classes.User now
# Address = Base.classes.address